In [1]:
import wandb
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns, matplotlib.pyplot as plt, operator as op


In [57]:
# api = wandb.Api(timeout=60)

In [58]:
# all_runs_data = {}
# agents = ['MDQN_modern', 'DQN_modern'] # DQN_modern
# environments = ['Qbert', 'BattleZone', 'DoubleDunk', 'NameThisGame', 'Phoenix']
# checkpoints = ['model_15250000', 'model_37250000', 'model_50000000']
# networks = [('nature', 0.001), 
#             ('dueling', 0.001),
#             ('impala_large:1', 0.0005),
#             ('impala_large:2', 0.0005),
#             ('impala_large:4', 0.0001),
#             # ('impalanextv2_large:1', 0.0001),
#             ('impalanextv2_large:2', 0.00001)]

# for agent in agents:
#     for environment in environments:
#         for checkpoint in checkpoints:
#             base_mae = None
#             for model, lr in networks:

#                 project_name = f'benchmark_final_{model}_{agent}_{environment}_{checkpoint}_0_{model}'.replace(':', '_')
#                 try:
#                     runs = api.runs('autoconvnext/' + project_name)
#                     all_runs_data[project_name] = []
#                     maes = []
#                     for run in runs:
                        
#                         if model == 'impalanextv2_large:2':
#                             # print(run.config['lr'])
#                             if run.config['lr'] == 0.0001:
#                                 continue
#                         all_runs_data[project_name].append(run.history())
#                 except Exception as e:
#                     print(project_name, e)

In [2]:
import numpy as np
# np.save('all_runs_data.npy', all_runs_data)
all_runs_data = np.load('all_runs_data.npy', allow_pickle=True).item()

In [3]:
def get_vals(agents, environments, checkpoints, networks):
    model_maes = {
        'dueling': [],
        'nature': [],
        'impala_large:1': [],
        'impala_large:2': [],
        'impala_large:4': [],
        # 'impalanextv2_large:1': [],
        'impalanextv2_large:2': [] # TODO check for the 2 the LR, there was a bug and we run it with two lrs
        }
    
    model_stds = {
        'dueling': [],
        'nature': [],
        'impala_large:1': [],
        'impala_large:2': [],
        'impala_large:4': [],
        # 'impalanextv2_large:1': [],
        'impalanextv2_large:2': [] # TODO check for the 2 the LR, there was a bug and we run it with two lrs
        }
    model_improvements = {
        'dueling': [],
        'nature': [],
        'impala_large:1': [],
        'impala_large:2': [],
        'impala_large:4': [],
        # 'impalanextv2_large:1': [],
        'impalanextv2_large:2': [] # TODO check for the 2 the LR, there was a bug and we run it with two lrs
        }
    i = 0
    for agent in agents:
        for environment in environments:
            for checkpoint in checkpoints:
                base_mae = None
                for model, lr in networks:

                    project_name = f'benchmark_final_{model}_{agent}_{environment}_{checkpoint}_0_{model}'.replace(':', '_')
                    # runs = api.runs('autoconvnext/' + project_name)
                    runs = all_runs_data[project_name]
                    maes = []
                    succ = 0
                    for run in runs:
                        try:
                            # if model == 'impalanextv2_large:2':
                            #     # print(run.config['lr'])
                            #     if run.config['lr'] == 0.0001:
                            #         continue
                            maes.append(run.iloc[29]['mae'])
                            succ += 1
                            if succ == 10:
                                break
                        except Exception as e:
                            print(project_name, e)
                    if succ < 10:
                        print(project_name, succ)
                    # print(project_name, pd.Series(maes).mean())
                    model_maes[model].append(pd.Series(maes).mean())
                    model_stds[model].append(pd.Series(maes).std())

                    if model == 'nature':
                        base_mae = pd.Series(maes).mean()
                    model_improvements[model].append((base_mae - pd.Series(maes).mean()) / base_mae * 100)
                    i += 1

                    if i % 10 == 0:
                        print(i)

    all = []
    for model, improvements in model_improvements.items():
        print(model, pd.Series(improvements).mean(), pd.Series(improvements).std())
        all.append(np.array(improvements))
    return model_maes, model_stds, model_improvements, all

    # sns.swarmplot(data=sorted_vals, size=6, edgecolor="black", linewidth=.9)

In [9]:
def plot(model_improvements, name):
    del model_improvements['nature']
    sorted_keys, sorted_vals = zip(*sorted(model_improvements.items(), key=op.itemgetter(1)))
    correct_order = ['dueling', 'impala_large:1', 'impala_large:2', 'impala_large:4', 'impalanextv2_large:2']
    sorted_keys = [correct_order.index(k) for k in sorted_keys]
    sorted_vals = list(sorted_vals)
    sorted_keys = list(sorted_keys)

    sorted_vals = [x for _, x in sorted(zip(sorted_keys, sorted_vals))]

    # almost verbatim from question
    sns.set(rc={'figure.figsize':(5,3)})

    sns.set(context='notebook', style='whitegrid')
    # sns.axlabel(xlabel="Groups", ylabel="Y-Axis", fontsize=16)
    sns.boxplot(data=sorted_vals, width=.4, orient='v')
    plt.xticks(range(0, len(sorted_keys)), ['Dueling', 'IMPALA:1', 'IMPALA:2', 'IMPALA:4', 'I-NeXt'])
    plt.xlabel('Model')
    plt.ylabel('Improvement (%)')
    plt.ylim(-10, 20)
    plt.savefig(f'figures/{name}.png', bbox_inches='tight')
    plt.savefig(f'figures/{name}.pdf', bbox_inches='tight')
    plt.clf()

In [4]:
all_agents = ['MDQN_modern', 'DQN_modern']
all_environments = ['Qbert', 'BattleZone', 'DoubleDunk', 'NameThisGame', 'Phoenix']
all_checkpoints = ['model_15250000', 'model_37250000', 'model_50000000']
networks = [('nature', 0.001), 
            ('dueling', 0.001),
            ('impala_large:1', 0.0005),
            ('impala_large:2', 0.0005),
            ('impala_large:4', 0.0001),
            ('impalanextv2_large:2', 0.00001)]

In [5]:
get_vals(all_agents, all_environments, all_checkpoints, networks)[2]

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
dueling -3.1341414195341954 6.678998431798878
nature 0.0 0.0
impala_large:1 6.511299590757182 3.814507252637409
impala_large:2 5.429128031749615 3.784677506251962
impala_large:4 7.247351333059283 4.282215034795758
impalanextv2_large:2 4.125647082936263 5.137107006351065


{'dueling': [-0.08050910528412523,
  -1.848807274965423,
  -1.0383411174604997,
  0.5402923591543178,
  0.437199465078546,
  -0.48598477679356045,
  -2.8851975416762308,
  -1.4872136573966546,
  -0.534768151419593,
  -1.1113240361484333,
  -1.2764271804627183,
  -2.7370784867016624,
  -34.427625812558475,
  -15.708012631675308,
  -8.16773422592798,
  -0.025622447379926323,
  -2.7781798360176513,
  0.6387717693280601,
  -0.30262812104177067,
  0.14712526323787128,
  0.1749104914170555,
  -2.5294231369718223,
  -3.084909800128784,
  -1.7650746229329444,
  -1.7301644933602582,
  -1.4220128090409372,
  -1.3799160349613175,
  -3.8676554055332817,
  -2.832806757140761,
  -2.455124471261601],
 'nature': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'impala_large:1': [5.4826139685351185,
  4.056255414566816,
  -0.8257326336368637,


In [10]:
plot(get_vals(all_agents, all_environments, all_checkpoints, networks)[2], 'all')

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
dueling -3.1341414195341954 6.678998431798878
nature 0.0 0.0
impala_large:1 6.511299590757182 3.814507252637409
impala_large:2 5.429128031749615 3.784677506251962
impala_large:4 7.247351333059283 4.282215034795758
impalanextv2_large:2 4.125647082936263 5.137107006351065


<Figure size 500x300 with 0 Axes>

In [64]:
for agent in all_agents:
    plot(get_vals([agent], all_environments, all_checkpoints, networks)[2], agent)

10
20
30
40
50
60
70
80
90
dueling -4.720768811615853 9.222759522188912
nature 0.0 0.0
impala_large:1 6.536613376605174 4.225367960165257
impala_large:2 5.706428914364098 3.6422048735867407
impala_large:4 6.947709187607632 4.478326099634526
impalanextv2_large:2 5.196753377432244 5.747992891560021
10
20
30
40
50
60
70
80
90
dueling -1.547514027452538 1.3966767242867604
nature 0.0 0.0
impala_large:1 6.485985804909186 3.5050167782009694
impala_large:2 5.151827149135132 4.029920850003387
impala_large:4 7.546993478510934 4.211501046432421
impalanextv2_large:2 3.0545407884402818 4.377998651399029


<Figure size 500x300 with 0 Axes>

In [65]:
for environment in all_environments:
    plot(get_vals(all_agents, [environment], all_checkpoints, networks)[2], environment)

10
20
30
dueling -0.8554480019632608 1.283662238455703
nature 0.0 0.0
impala_large:1 3.3378076882551952 2.4737679766793392
impala_large:2 3.6276023778762974 2.5129719652922797
impala_large:4 5.306792181879019 1.456780085793692
impalanextv2_large:2 -3.680163609359005 1.9223067490070678
10
20
30
dueling 0.08515244684207658 0.40485999466908984
nature 0.0 0.0
impala_large:1 8.618802972258129 0.7219553064096135
impala_large:2 9.253393748364529 0.8985931128956193
impala_large:4 9.957539340129733 1.3135819160034348
impalanextv2_large:2 4.331739464979645 0.7290605966726746
10
20
30
dueling -2.0477644850876717 0.9687361126743478
nature 0.0 0.0
impala_large:1 9.601742164761637 0.3987074407405923
impala_large:2 8.123596953629464 0.736431874007008
impala_large:4 10.085571765843712 0.2562720122625462
impalanextv2_large:2 6.779445999872094 1.514575992226166
10
20
30
dueling -1.6094871734458878 0.5887223794784314
nature 0.0 0.0
impala_large:1 1.8076536769213682 1.4244094389236697
impala_large:2 0.566

<Figure size 500x300 with 0 Axes>

In [66]:
for checkpoint in all_checkpoints:
    plot(get_vals(all_agents, all_environments, [checkpoint], networks)[2], checkpoint)

10
20
30
40
50
60
dueling -4.64198577408 10.562414247091642
nature 0.0 0.0
impala_large:1 7.79559324065909 3.7285690363535564
impala_large:2 6.505499140006859 2.762450060536319
impala_large:4 8.548344810302762 3.7002864498901924
impalanextv2_large:2 3.940798632154471 4.698752527232049
10
20
30
40
50
60
dueling -2.9854045218511818 4.623239290555495
nature 0.0 0.0
impala_large:1 5.898006382052356 3.3918164193813083
impala_large:2 5.396365694088837 3.932810758898885
impala_large:4 7.185167912402288 3.6531841575207844
impalanextv2_large:2 4.492106141414633 6.248712027938532
10
20
30
40
50
60
dueling -1.775033962671404 2.489583894446774
nature 0.0 0.0
impala_large:1 5.840299149560094 4.327507460503758
impala_large:2 4.385519261153151 4.5340548414549335
impala_large:4 6.008541276472805 5.334705263162607
impalanextv2_large:2 3.9440364752396846 4.866776842618435


<Figure size 500x300 with 0 Axes>